In [1]:
import requests
from bs4 import BeautifulSoup
import re

import pandas as pd

def recuperer_contenu(url, nom_classe):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Erreur lors de la requête : {e}")
        return []
    
    soup = BeautifulSoup(response.text, 'html.parser')
    balises = soup.find_all(class_=nom_classe)
    
    return balises

In [45]:
# Recuperation des url 

import json
def read_json_file(file_path):
    try:
        # Ouverture et lecture du fichier JSON
        with open(file_path, 'r', encoding='utf-8') as file:
            # Conversion du contenu JSON en dictionnaire Python
            data = json.load(file)
            return data
    except FileNotFoundError:
        print(f"Erreur: Le fichier {file_path} n'existe pas")
        return None
    except json.JSONDecodeError:
        print(f"Erreur: Le fichier {file_path} n'est pas un JSON valide")
        return None
    except Exception as e:
        print(f"Une erreur inattendue s'est produite: {str(e)}")
        return None

def get_ship(data, name):
    return data[name]
    

def get_url(data,name):
    url = get_ship(data,name)['pledge_url']
    return url


data = read_json_file('shipDB_All.json')

# creation d une liste contenant toutes les url 
 
def list_ship(data):
    Flight_ready=[]
    In_Concept = []
    for i in data:
        if 'pledge_url' in data[i].keys():
            Flight_ready.append(i)
        else : 
            In_Concept.append(i)
    return Flight_ready,In_Concept 
    

In [46]:
Flight_ready = [get_ship(data,i)['pledge_url']  for i in list_ship(data)[0]] 

Flight_ready[16]

'https://robertsspaceindustries.com/pledge/ships/rsi-apollo/Apollo-Medivac'

In [31]:
from collections import defaultdict

def fusion_complete(l):
    lst = lambda:list(range(0,0))
    resultat = defaultdict(lst)
    for d in l:
        for cle, valeur in d.items():
            resultat[cle].append(valeur)

    return dict(resultat)


def noms(list = Flight_ready):
    nom = []
    for i in list:
        nom.append(i.split('/')[-1])
    return nom

noms()

['100i',
 '125a',
 '135c',
 '300i',
 '315p',
 '325a',
 '350r',
 '400i',
 '600i-Explorer',
 '600i-Touring',
 '85X',
 '890-Jump',
 'A1-Spirit',
 'A2-Hercules',
 'Anvil-Ballista-Dunestalker',
 'Anvil-Ballista-Snowblind',
 'Apollo-Medivac',
 'Apollo-Triage',
 'Ares-Inferno',
 'Ares-Ion',
 'Argo-Mole-Carbon-Edition',
 'Argo-Mole-Talus-Edition',
 'Arrastra',
 'Arrow',
 'ATLS',
 'Aurora-CL',
 'Aurora-ES',
 'Aurora-LN',
 'Aurora-LX',
 'Aurora-MR',
 'Avenger-Stalker',
 'Avenger-Titan',
 'Avenger-Titan-Renegade',
 'Avenger-Warlock',
 'Ballista',
 'Blade',
 'Buccaneer',
 'C1-Spirit',
 'C2-Hercules',
 'C8R-Pisces',
 'C8X-Pisces-Expedition',
 'Carrack',
 'Carrack-Expedition',
 'Caterpillar',
 'Caterpillar-Best-In-Show-Edition',
 'Caterpillar-Pirate-Edition',
 'Centurion',
 'Constellation-Andromeda',
 'Constellation-Aquila',
 'Constellation-Phoenix',
 'Constellation-Phoenix-Emerald',
 'Constellation-Taurus',
 'Corsair',
 'Crucible',
 'CSV-SM',
 'Cutlass-Black',
 'Cutlass-Black-Best-In-Show-Edition',

In [86]:
#creation du dataframe

def DataBase():
    nomclasse = "l-sheet__block"
    dict = {}
    liste_dict=[]
    for i in Flight_ready[:]:
        dict = {}
        for e in recuperer_contenu(i, nomclasse):
            e = e.text.replace("\n", "|").replace("\t", "")
            e = e.split("|")
            n = []
            for i in e:
                i = re.sub(r'\s+', ' ', i).strip()
                if len(i)!=0:
                    n.append(i)
            for i in range(0, len(n)-2, 2):
                dict[n[i]] = n[i+1]
        print(dict)
        #dict=fusioncomplete(dict,dict)
        liste_dict.append(dict)
    dict=fusion_complete( liste_dict)

    return dict

DB = DataBase()

{'Focus': 'Starter / Touring', 'Length': '19.3m', 'Beam': '11.0m', 'Height': '4.0m', 'Size': 'Small', 'Cargo Capacity': '2', 'SCM Speed': '260 m/s', 'Afterburner Speed': '1,425 m/s', 'Min Crew': '1', 'Pitch Max': '- deg/s', 'Yaw Max': '- deg/s', 'Roll Max': '- deg/s', 'X-Axis Acceleration': '- m/s/s', 'Y-Axis Acceleration': '- m/s/s'}
{'Focus': 'Starter / Light Fighter', 'Length': '19.3m', 'Beam': '11.0m', 'Height': '4.0m', 'Size': 'Small', 'Cargo Capacity': '2', 'SCM Speed': '260 m/s', 'Afterburner Speed': '1,425 m/s', 'Min Crew': '1', 'Pitch Max': '- deg/s', 'Yaw Max': '- deg/s', 'Roll Max': '- deg/s', 'X-Axis Acceleration': '- m/s/s', 'Y-Axis Acceleration': '- m/s/s'}
{'Focus': 'Starter / Light Freight', 'Length': '19.3m', 'Beam': '11.0m', 'Height': '4.0m', 'Size': 'Small', 'Cargo Capacity': '6', 'SCM Speed': '260 m/s', 'Afterburner Speed': '1,425 m/s', 'Min Crew': '1', 'Pitch Max': '- deg/s', 'Yaw Max': '- deg/s', 'Roll Max': '- deg/s', 'X-Axis Acceleration': '- m/s/s', 'Y-Axis Acc

KeyboardInterrupt: 

In [2]:
ship_db = pd.read_json("shipDB_All.json").T
ship_db = ship_db.drop(['description','pledge_url','chassis_id','updated_at','production_note','loaner','id','size_class','slug','version','class_name','msrp'], axis = 1)
Sizes = dict(ship_db['sizes'])
Speed = dict(ship_db['speed'])

def Speeds(D=Speed):
    scm={}
    maxi = {}
    
    for k,v in Speed.items():
        if str(v) == 'None':
            scm[k]=maxi[k]='None'
        else : 
            for i,j in v.items():
                if i == 'scm':
                    scm[k]= j
                elif i == 'max':
                    maxi[k] = j  
    return maxi,scm

In [94]:
Speeds()

({'100i': 1425,
  '125a': 1425,
  '135c': 1425,
  '300i': 1425,
  '315p': 1425,
  '325a': 1425,
  '350r': 1425,
  '400i': 1000,
  '600i Explorer': 975,
  '600i Touring': 975,
  '85X': 1250,
  '890 Jump': 915,
  'A1 Spirit': 1130,
  'A2 Hercules': 950,
  'Anvil Ballista Dunestalker': 25,
  'Anvil Ballista Snowblind': 25,
  'Apollo Medivac': None,
  'Apollo Triage': None,
  'Ares Inferno': 1100,
  'Ares Ion': 1100,
  'Argo Mole Carbon Edition': 960,
  'Argo Mole Talus Edition': 960,
  'Arrastra': None,
  'Arrow': 1210,
  'ATLS': None,
  'Aurora CL': 1200,
  'Aurora ES': 1200,
  'Aurora LN': 1200,
  'Aurora LX': 1200,
  'Aurora MR': 1200,
  'Avenger Stalker': 1425,
  'Avenger Titan': 1425,
  'Avenger Titan Renegade': 1425,
  'Avenger Warlock': 1425,
  'Ballista': 25,
  'Blade': 1200,
  'Buccaneer': 1400,
  'C1 Spirit': 1130,
  'C2 Hercules': 950,
  'C8 Pisces': 1150,
  'C8R Pisces': 1150,
  'C8X Pisces Expedition': 1150,
  'Carrack': 1050,
  'Carrack Expedition': 1050,
  'Caterpillar': 90

In [10]:
vaisseaux =  recuperer_contenu('https://scfocus.org/ship-sale-rental-locations-history/',nom_classe= 'column-1')
prix= recuperer_contenu('https://scfocus.org/ship-sale-rental-locations-history/',nom_classe= 'column-2')

def url_to_list(D):
    L =[]
    for e in D:
        e = e.text.replace("\n", "|").replace("\t", "")
        e = e.split("|")
        L += e
    return L[1:]

prix_doublons= [i for i in zip(url_to_list(vaisseaux),url_to_list(prix))]
prix = dict(set(prix_doublons))

In [ ]:
recuperer_contenu('https://robertsspaceindustries.com/en/pledge/ships/origin-100/100i',nom_classe='statbox title') 

[]

In [35]:
input_prices = prix
#del  input_prices['Ship']

def clean_price(price_str):
    # Remove any parentheses and their content
    if '(' in price_str:
        price_str = price_str[:price_str.index('(')].strip()
    return float(price_str.replace(',', ''))
    # Remove commas and convert to float
    #return float(price_str.replace(',', ''))
    
converted_prices = {k: clean_price(v) for k, v in input_prices.items()}
converted_prices

{'X1 Velocity': 119020.0,
 'Cyclone': 103950.0,
 'Cutter Scout': 705600.0,
 'Hammerhead': 47958752.0,
 'Arrow': 39690.0,
 '300i': 41277.0,
 'Nox': 27740.0,
 '600i': 816952.0,
 'Mustang Alpha': 642600.0,
 'Greycat ROC': 103950.0,
 'Gatac Syulen': 2778300.0,
 '890 Jump': 65365200.0,
 'Avenger Warlock': 84334.0,
 'Dragonfly': 342720.0,
 'Sabre Peregrine': 4048380.0,
 'Anvil Arrow': 39690.0,
 'Constellation Phoenix': 247506.0,
 'Aurora CL': 1020600.0,
 'Mustang Beta': 655200.0,
 'Retaliator': 7938000.0,
 'Cutlass Black': 61387.0,
 'Corsair': 6552000.0,
 'Ares Star Fighter Inferno': 4725000.0,
 'Constellation Andromeda': 203212.0,
 'Tumbril Cyclone': 2165.0,
 'Storm': 453600.0,
 'M50 Interceptor': 44982.0,
 'Cutlass Red': 2857680.0,
 '350r': 3748500.0,
 '325a': 1852200.0,
 'Cutter': 635040.0,
 'Hull A': 1701000.0,
 'Cyclone RN': 122850.0,
 'M2 Starlifter': 29484000.0,
 'Terrapin': 5433120.0,
 'Aurora LN': 22679.0,
 'Nomad': 44724.0,
 'Mercury Star Runner': 12285000.0,
 'Gladius Valiant': 29

In [4]:
stardle_db = pd.read_json('shipDB_Stardle.json').T
stardle_db = stardle_db.drop('description',axis = 1)
stardle_db

,name,dimensions,cargo_capacity,mass,crew,speed,manufacturer,type,status,role,price,release_date
100i,100i,19,2,48143,1,1425,Origin Jumpworks,multi,flight-ready,"{'0': 'Starter', '1': 'Touring'}",50.0,2020
125a,125a,19,2,48143,1,1425,Origin Jumpworks,multi,flight-ready,"{'0': 'Starter', '1': 'Light Fighter'}",60.0,2020
135c,135c,19,6,48143,1,1425,Origin Jumpworks,multi,flight-ready,"{'0': 'Starter', '1': 'Light Freight'}",65.0,2020
300i,300i,27,8,79677,1,1425,Origin Jumpworks,multi,flight-ready,"{'0': 'Touring', '1': 'Luxury'}",60.0,2014
315p,315p,27,12,80592,1,1425,Origin Jumpworks,exploration,flight-ready,{'0': 'Pathfinder'},65.0,2014
...,...,...,...,...,...,...,...,...,...,...,...,...
X1 Force,X1 Force,1.55,0,4730,1,569,Origin Jumpworks,exploration,flight-ready,{'0': 'Combat'},50.0,2023
X1 Velocity,X1 Velocity,1.55,0,4730,1,611,Origin Jumpworks,competition,flight-ready,{'0': 'Racing'},45.0,2023
Zeus CL Mk II,Zeus CL Mk II,46,128,295000,3,1000,Roberts Space Industries,transport,flight-ready,{'0': 'Medium Freight'},150.0,2024
Zeus ES Mk II,Zeus ES Mk II,46,32,295000,3,1050,Roberts Space Industries,exploration,flight-ready,{'0': 'Expedition'},150.0,2024


In [32]:
#stardle_db = stardle_db.drop(['speed'],axis=1)
stardle_db = stardle_db.assign(scm = Speeds()[1],
                         max = Speeds()[0])

stardle_db = stardle_db.assign(price_ingame = converted_prices)

In [33]:
stardle_db

,name,dimensions,cargo_capacity,mass,crew,manufacturer,type,status,role,price,release_date,scm,max,price_ingame
100i,100i,19,2,48143,1,Origin Jumpworks,multi,flight-ready,"{'0': 'Starter', '1': 'Touring'}",50.0,2020,260.0,1425.0,1146600.0
125a,125a,19,2,48143,1,Origin Jumpworks,multi,flight-ready,"{'0': 'Starter', '1': 'Light Fighter'}",60.0,2020,260.0,1425.0,1587600.0
135c,135c,19,6,48143,1,Origin Jumpworks,multi,flight-ready,"{'0': 'Starter', '1': 'Light Freight'}",65.0,2020,260.0,1425.0,1490580.0
300i,300i,27,8,79677,1,Origin Jumpworks,multi,flight-ready,"{'0': 'Touring', '1': 'Luxury'}",60.0,2014,260.0,1425.0,41277.0
315p,315p,27,12,80592,1,Origin Jumpworks,exploration,flight-ready,{'0': 'Pathfinder'},65.0,2014,260.0,1425.0,1490580.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X1 Force,X1 Force,1.55,0,4730,1,Origin Jumpworks,exploration,flight-ready,{'0': 'Combat'},50.0,2023,207.0,569.0,132300.0
X1 Velocity,X1 Velocity,1.55,0,4730,1,Origin Jumpworks,competition,flight-ready,{'0': 'Racing'},45.0,2023,222.0,611.0,119020.0
Zeus CL Mk II,Zeus CL Mk II,46,128,295000,3,Roberts Space Industries,transport,flight-ready,{'0': 'Medium Freight'},150.0,2024,NaN,NaN,NaN
Zeus ES Mk II,Zeus ES Mk II,46,32,295000,3,Roberts Space Industries,exploration,flight-ready,{'0': 'Expedition'},150.0,2024,NaN,NaN,NaN


In [34]:
stardle_db.to_csv('stradle_db.csv', index=False)